In [13]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2
from cvzone.PoseModule import PoseDetector
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle 
import pandas as pd
from datetime import datetime, date
import numpy as np

date = date.today()
date = date.strftime("%B %d, %Y")

path = "C:/Users/Ejay Olivar/Desktop/PoseClassification/Models/"


In [14]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [15]:
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(path+"MobileNetSSD_deploy.prototxt.txt", path+"MobileNetSSD_deploy.caffemodel")

[INFO] loading model...


In [16]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [17]:
with open(path+'pose_classification.pkl', 'rb') as f:
    model = pickle.load(f)

In [18]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [22]:
def RTPoseClassification(image,draw=True):
    
     # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        imageRGB.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(imageRGB)
        # print(results.face_landmarks)

        if results.pose_landmarks:
                # 4. Pose Detections
            if draw:
                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                        )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row + face_row

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            

            body_language_class = "SOuspicious Movements "
  
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            if((round(body_language_prob[np.argmax(body_language_prob)],2))>.99):        
                print(body_language_class)
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                return True
            else:
                return False
                
        except:
            return False

In [23]:
PosibleCheatingFolder = "PosibleCheatingFolder"
if not os.path.exists(PosibleCheatingFolder):
    os.makedirs(PosibleCheatingFolder)

PosibleCheatingFolderPerday = "PosibleCheatingFolder/PosibleCheatingFolderPerday/" + date
if not os.path.exists(PosibleCheatingFolderPerday):
    os.makedirs(PosibleCheatingFolderPerday)

In [24]:
# initialize the video stream, allow the cammera sensor to warmup,
# and initialize the FPS counter
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(1)

fps = FPS().start()
detector = PoseDetector()

a=0

# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	ret, frame = cap.read()
	frame = imutils.resize(frame, width=1200)

	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
		0.007843, (300, 300), 127.5)

	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()
	
	
	# loop over the detections
	for i in np.arange(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the prediction
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the `confidence` is
		# greater than the minimum confidence
		if confidence > 0.4:
			# extract the index of the class label from the
			# `detections`, then compute the (x, y)-coordinates of
			# the bounding box for the object
			idx = int(detections[0, 0, i, 1])
			if idx == 15:
				box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			
				(startX, startY, endX, endY) = box.astype("int")
				#print(startX, startY, endX, endY)
					
				Y1,Y2,X1,X2= startY ,endY ,startX, endX
				#print(X1,Y1,X2,Y2)
				if (Y1 < 0 ):
					Y1 = 0
				if (X1 < 0 ):
					X1 = 0
				if (Y2 < 0 ):
					Y2 = 0
				if (X2 < 0 ):
					X2 = 0
				frame_crop = frame[Y1:Y2, X1:X2]
				#print(frame_crop)
				
				label = "{}: {:.2f}%".format(CLASSES[idx],confidence * 100)

				alert = RTPoseClassification(frame_crop)

				if alert == True:
					al = cv2.resize(frame, (960, 540), interpolation = cv2.INTER_AREA)
					cv2.imwrite(PosibleCheatingFolderPerday+"/"+"PossibleCheatingPics-"+str(a)+".jpg",al)
					#cv2.imshow("Alert", al)
					a = a + 1
						
	# show the output frame
	cv2.imshow("frame",frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

	# update the FPS counter
	fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
cv2.destroyAllWindows()


[INFO] starting video stream...
[INFO] elapsed time: 10.02
[INFO] approx. FPS: 5.19
